In [2]:
pip install tensorflow


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ------------------------- -------------- 30.7/48.7 kB ? eta -:--:--
     -------------------------------------- 48.7/48.7 kB 620.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.2 MB 2.6 MB/s eta 0:02:28
   ---------------------------------------- 0.3/390.2 MB 2.6 MB/s eta 0:02:30
   ---------------------------------------- 0.5/390.2 MB 3.7 MB/s eta 0:01:46
   ---------------------------------------- 0.8/390.2 MB 4.6 MB/s eta 0:01:25
   ---------------------------------------- 0.8/390.2 MB 4.6 MB/s eta 0:01:25
   ---------------------------------------- 1.1/390.2 MB 4.2 MB/s eta 0:01:33
   ---------------------------------------- 1.4/390.2 MB 4.2 MB/s eta 0:01:34
   ---------------------------------------- 2.1/390.2 MB 5.7 MB/s eta 0:01:09
   ---------------------------------------- 2.2/390.2 MB 5.8 MB/s eta 0:01:07
  

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

# Preprocess MNIST data
x_train_mnist = x_train_mnist.reshape(x_train_mnist.shape[0], 28, 28, 1).astype('float32') / 255
x_test_mnist = x_test_mnist.reshape(x_test_mnist.shape[0], 28, 28, 1).astype('float32') / 255
y_train_mnist = to_categorical(y_train_mnist, 10)
y_test_mnist = to_categorical(y_test_mnist, 10)

# Load A-Z dataset (assuming it's saved as a CSV file)
az_dataset = pd.read_csv('A_Z Handwritten Data.csv', header=None)  # Assuming no header in CSV file

# Separate features and labels
X_az = az_dataset.drop(0, axis=1).values.astype('float32') / 255  # Drop column '0' as label
y_az = az_dataset[0].values  # Use column '0' as label

# Convert labels to categorical one-hot encoding
label_bin = LabelBinarizer()
y_az = label_bin.fit_transform(y_az)

# Split the data into training and testing sets
x_train_az, x_test_az, y_train_az, y_test_az = train_test_split(X_az, y_az, test_size=0.2, random_state=42)

# Define the first CNN model for MNIST digits recognition
def mnistModel1():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(15, (3, 3), activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(10, activation="softmax"))

    # Compilation
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Define the second CNN model for MNIST digits recognition
def mnistModel2():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(15, (3, 3), activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(10, activation="softmax"))

    # Compilation
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Instantiate and compile the first MNIST model
mnist_model1 = mnistModel1()

# Train the first MNIST model
mnist_model1_history = mnist_model1.fit(x_train_mnist, y_train_mnist, epochs=5, batch_size=32, validation_data=(x_test_mnist, y_test_mnist))

# Save the first MNIST model
mnist_model1.save('mnist_model1.h5')

# # Evaluate the first MNIST model
# mnist1_loss, mnist1_acc = mnist_model1.evaluate(x_test_mnist, y_test_mnist)
# print(f"MNIST Model 1 Accuracy: {mnist1_acc*100:.2f}%")

# Instantiate and compile the second MNIST model
mnist_model2 = mnistModel2()

# Train the second MNIST model
mnist_model2_history = mnist_model2.fit(x_train_mnist, y_train_mnist, epochs=10, batch_size=200, validation_data=(x_test_mnist, y_test_mnist))

# Save the second MNIST model
mnist_model2.save('mnist_model2.h5')

# # Evaluate the second MNIST model
# mnist2_loss, mnist2_acc = mnist_model2.evaluate(x_test_mnist, y_test_mnist)
# print(f"MNIST Model 2 Accuracy: {mnist2_acc*100:.2f}%")

# Define a Dense Neural Network model for A-Z alphabet recognition
az_model = Sequential([
    Dense(256, activation='relu', input_shape=(784,)),
    Dense(128, activation='relu'),
    Dense(26, activation='softmax')  # 26 classes for A-Z
])

az_model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

# Train the A-Z model
az_model_history = az_model.fit(x_train_az, y_train_az, epochs=10, batch_size=32, validation_data=(x_test_az, y_test_az))

# Save the A-Z mode
az_model.save('az_model.h5')

# Evaluate the A-Z model
az_loss, az_acc = az_model.evaluate(x_test_az, y_test_az)
print(f"Accuracy: {az_acc*100:.2f}%")

# Plotting function
def plot_history(history, model_name):
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.tight_layout()
    plt.show()

# # Plot for first MNIST model
# plot_history(mnist_model1_history, 'MNIST Model 1')

# # Plot for second MNIST model
# plot_history(mnist_model2_history, 'MNIST Model 2')

# Plot 
plot_history(az_model_history, 'HCR')


C:\Users\library\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8681 - loss: 0.4068 - val_accuracy: 0.9832 - val_loss: 0.0524
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9767 - loss: 0.0737 - val_accuracy: 0.9836 - val_loss: 0.0457
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9840 - loss: 0.0518 - val_accuracy: 0.9869 - val_loss: 0.0399
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9865 - loss: 0.0433 - val_accuracy: 0.9905 - val_loss: 0.0289
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9876 - loss: 0.0384 - val_accuracy: 0.9906 - val_loss: 0.0303


Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.7497 - loss: 0.8162 - val_accuracy: 0.9770 - val_loss: 0.0730
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9695 - loss: 0.1020 - val_accuracy: 0.9817 - val_loss: 0.0529
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9770 - loss: 0.0755 - val_accuracy: 0.9871 - val_loss: 0.0377
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9826 - loss: 0.0571 - val_accuracy: 0.9877 - val_loss: 0.0356
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9847 - loss: 0.0463 - val_accuracy: 0.9909 - val_loss: 0.0283
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9866 - loss: 0.0420 - val_accuracy: 0.9905 - val_loss: 0.0282
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9877 - loss: 0.0373 - val_accuracy: 0.9916 - val_loss: 0.0257
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9895 - loss: 0.0330 - val_accu

C:\Users\library\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 16), output.shape=(None, 26)